In [ ]:
"""
## 8. 强化学习：价值迭代 (Value Iteration)

### 问题：网格世界寻路
- 3×4网格，含有墙壁和陷阱
- 目标：找到最优策略最大化累积奖励

### 贝尔曼最优方程
V*(s) = max_a [R(s,a) + γ * V*(s')]

### 价值迭代算法
1. 初始化所有状态的价值为0
2. 对每个状态，计算所有动作的Q值
3. 取Q值最大的作为新的状态价值
4. 重复直到收敛（价值变化<阈值）

### 最优策略提取
π*(s) = argmax_a [R(s,a) + γ * V*(s')]

### 应用场景
- 机器人路径规划
- 游戏AI
- 资源分配等
"""

import sys

class GridWorld:
    def __init__(self):
        # 定义网格世界的形状
        self.shape = (3, 4)
        # 定义终止状态及其奖励
        self.terminal_states = {(0, 3): 1, (1, 3): -1}
        # 定义墙壁位置
        self.wall_states = [(1, 1)]
        # 定义所有状态（不包括墙壁）
        self.states = []
        for r in range(self.shape[0]):
            for c in range(self.shape[1]):
                if (r, c) not in self.wall_states:
                    self.states.append((r, c))
        # 定义动作
        self.actions = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        # 定义移动的固定成本
        self.step_reward = -0.04
        # 折扣因子
        self.gamma = 1.0

    def get_next_state_and_reward(self, state, action):
        """
        确定性环境：给定状态和动作，返回下一状态和奖励
        """
        if state in self.terminal_states:
            return state, 0

        move = self.actions[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        
        # 边界和墙壁检查
        r, c = next_state
        if r < 0 or r >= self.shape[0] or c < 0 or c >= self.shape[1] or next_state in self.wall_states:
            next_state = state
        
        # 奖励
        if next_state in self.terminal_states:
            reward = self.terminal_states[next_state]
        else:
            reward = self.step_reward
            
        return next_state, reward

def value_iteration(env, theta=1e-4):
    """
    执行价值迭代算法
    
    Args:
        env: GridWorld环境
        theta: 收敛阈值
        
    Returns:
        tuple: (最优价值函数, 最优策略)
    """
    # 初始化价值函数
    V = {s: 0 for s in env.states}
    
    iteration = 0
    while True:
        iteration += 1
        delta = 0
        
        # 对所有状态更新价值
        for s in env.states:
            if s in env.terminal_states:
                continue
            
            v_old = V[s]
            action_values = []
            
            # 计算所有动作的Q值
            for action in env.actions:
                next_state, reward = env.get_next_state_and_reward(s, action)
                q_value = reward + env.gamma * V[next_state]
                action_values.append(q_value)
            
            # 贝尔曼最优方程：取最大Q值
            V[s] = max(action_values)
            delta = max(delta, abs(v_old - V[s]))
        
        print(f"Iteration {iteration}, Delta: {delta:.6f}")

        # 收敛检查
        if delta < theta:
            break
    
    # 提取最优策略
    pi_star = {}
    for s in env.states:
        if s in env.terminal_states:
            continue
        
        best_action = None
        max_q_value = -float('inf')
        
        for action in env.actions:
            next_state, reward = env.get_next_state_and_reward(s, action)
            q_value = reward + env.gamma * V[next_state]
            if q_value > max_q_value:
                max_q_value = q_value
                best_action = action
        
        pi_star[s] = best_action
    
    return V, pi_star

# 测试（需在外部调用）
# if __name__ == "__main__":
#     env = GridWorld()
#     V, pi = value_iteration(env)
